# GDP vs Measles Vaccination

In this project we seek to analyze the GDP for selected countries depending on the child measles vaccination rates. 

Imports and set magics:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


We have imported data in cvs-files from the OECD Databank. 

**The 1st dataset:**
The first dataset is an overview of GDP per capita for 7 countries in the world from 2018 to 2021. The included variables are year, GDP per capita and country. 

**The 2nd dataset:**
The second dataset is an overview of child vaccination rates for 7 countries in the world from 2018 to 2021. The included variables are year, child measles vaccination rates and country. 

# Importing and cleaning the first dataset

In [ ]:
#import data
filename = 'GDP2.xlsx'
pd.read_excel(filename).head(7)

gdp = pd.read_excel(filename, skiprows=2)
gdp.head(7)


In [ ]:
#Removing the first columns
drop_these = ['Unnamed: ' + str(num) for num in range(2)]
gdp.drop(drop_these, axis=1, inplace=True)
gdp.head(7)

In [ ]:
#Renaming the columns
gdp.rename(columns = {'Unnamed: 2':'country'}, inplace=True)
gdp.rename(columns = {2018:'gdp2018'}, inplace=True)
gdp.rename(columns = {2019:'gdp2019'}, inplace=True)
gdp.rename(columns = {2020:'gdp2020'}, inplace=True)
gdp.rename(columns = {2021:'gdp2021'}, inplace=True)
gdp.head(7)

In [ ]:
#Resetting the index
gdp.reset_index(inplace = True, drop = True)
gdp.iloc[0:7,:]

In [ ]:
#Changing from wide to long
gdp_long = pd.wide_to_long(gdp, stubnames='gdp', i='country', j='year')
gdp_long.head(7)

In [ ]:
gdp_long = gdp_long.reset_index()
gdp_long.loc[gdp_long.country == 'China', :]

**Plotting the first dataset**

In [ ]:
gdp_china = gdp_long.loc[gdp_long.country == 'China', :]
gdp_china.plot(x='year', y='gdp', legend=False);
plt.title("GDP pr. capita in China")
plt.show()

# Importing and cleaning the second dataset

In [ ]:
#import data
filename = 'vac.xlsx'
pd.read_excel(filename).head(7)

vac = pd.read_excel(filename)
vac.head(7)

In [ ]:
#Renaming the columns
vac.rename(columns = {'Unnamed: 0':'country'}, inplace=True)
vac.rename(columns = {2018:'vac2018'}, inplace=True)
vac.rename(columns = {2019:'vac2019'}, inplace=True)
vac.rename(columns = {2020:'vac2020'}, inplace=True)
vac.rename(columns = {2021:'vac2021'}, inplace=True)
vac.head(7)

In [ ]:
#Resetting the index
vac.reset_index(inplace = True, drop = True)
vac.iloc[0:7,:]

In [ ]:
#Changing from wide to long
vac_long = pd.wide_to_long(vac, stubnames='vac', i='country', j='year')
vac_long.head(7)

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Plotting the vaccine rates for China in a static figure**

In [ ]:
vac_long = vac_long.reset_index()
vac_long.loc[vac_long.country == 'China', :]

vac_china = vac_long.loc[vac_long.country == 'China', :]
vac_china.plot(x='year', y='vac', legend=False);
plt.title("Measles vaccination rates for children in China")
plt.show()

**Interactive plot** :

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now we combine our loaded data sets. 

In [ ]:
inner = pd.merge(gdp,vac, how='left' ,on=['country'])
print(inner)

In [ ]:
#Changing from wide to long
inner_long = pd.wide_to_long(inner, stubnames=['gdp', 'vac'], i='country', j='year')
inner_long.head(7)

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.